In [ ]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import numpy as np
import time
from datetime import date,timedelta
import base64
import pytesseract
import cv2

In [ ]:
#Config
ct = 0
PATH = ""
tesseractPath = ""
lastXDays = ""
searchList = []
sitePath = "https://rarbgcore.org"

f = open("Config.txt", "r")
while(True):
    ct+=1
    #read next line
    line = f.readline()
    #if line is empty, you are done with all lines in the file
    if not line:
        break
    #you can access the line
    if ct == 1:
        PATH = line.strip()
    if ct == 2:
        tesseractPath = line.strip()
    if ct == 3:
        lastXDays = line.strip()
    if ct == 4:
        searchList = line.strip().split(",")
#close file
f.close

In [ ]:
driver = webdriver.Chrome(PATH)
time.sleep(5)

In [ ]:
# get past bot checker
driver.get(sitePath + "/torrents.php")
time.sleep(10)
# driver.get("https://rarbg.to/threat_defence.php?defence=1")

In [ ]:
# get captcha image
img_base64 = driver.execute_script("""
    var ele = arguments[0];
    var cnv = document.createElement('canvas');
    cnv.width = 215; cnv.height = 80;
    cnv.getContext('2d').drawImage(ele, 0, 0);
    return cnv.toDataURL('image/jpeg').substring(22);    
    """, driver.find_element_by_xpath("/html/body/form/div/div/table[1]/tbody/tr[2]/td[2]/img"))
with open(r"image.jpg", 'wb') as f:
    f.write(base64.b64decode(img_base64)) 

In [ ]:
# load and decode captcha image
pytesseract.pytesseract.tesseract_cmd = tesseractPath
img = cv2.imread('image.jpg')
text = pytesseract.image_to_string(img)
text = text.rsplit(' ', 1)[0]

In [ ]:
# enter captcha text
inputElement = driver.find_element_by_id("solve_string")
inputElement.send_keys(text)
inputElement.send_keys(Keys.ENTER)

In [ ]:
def range1(start, end):
    return range(start, end+1)

def getTablefrom(urlSearch,pageNum):
    driver.get(sitePath + "/torrents.php?search="+ urlSearch + "&order=filename&by=DESC&page=" + str(pageNum))
    driver.implicitly_wait(5)
    data =[]
    content = driver.find_elements_by_class_name('lista2')
    for element in content:
        data.append(element.text)

    # separete into genre and no genre tables
    df = pd.DataFrame(data,columns=['tableinfo'])
    df["genre"] = df["tableinfo"].str.contains(r'\n')
    df_genre = df[df.genre == True]
    df_NOgenre = df[df.genre == False]

    # get genre table
    genre_df = pd.DataFrame()
    df_genre = df_genre["tableinfo"].str.split(r"\n", expand = True)
    df_genre["newGenre"] = df_genre[1].str.startswith(r"20")
    genre_df["Name"] = df_genre[0]

    df_genre_1 = df_genre[df_genre.newGenre == True]
    df_genre_1 = df_genre_1.drop(columns=['newGenre'])

    df_genre_2 = df_genre[df_genre.newGenre == False]
    df_genre_2 = df_genre_2[1].str.split(' (?=20..[-]..[-])',expand = True)

    df_genre = df_genre_1.append(df_genre_2)
    df_genre = df_genre[1].str.split(r' ', expand = True)
    df_genre_merged = genre_df.merge(df_genre, how='inner', left_index=True, right_index=True)
    df_genre_merged = df_genre_merged.rename(columns={0:"Date",1:"Time",2:"Size",3:"Size_Lable",4:"Seeders",5:"Leachers",6:"Comments",7:"Uploader"})

    # get no genre table
    if df_NOgenre.empty:
        pass
    else:
        NOgenre_df = pd.DataFrame()
        df_NOgenre = df_NOgenre["tableinfo"].str.split(' (?=20..[-]..[-])',expand = True)
        NOgenre_df["Name"] = df_NOgenre[0]
        df_NOgenre = df_NOgenre[1].str.split(r' ', expand = True)
        df_NOgenre_merged = NOgenre_df.merge(df_NOgenre, how='inner', left_index=True, right_index=True)
        df_NOgenre_merged = df_NOgenre_merged.rename(columns={0:"Date",1:"Time",2:"Size",3:"Size_Lable",4:"Seeders",5:"Leachers",6:"Comments",7:"Uploader"})

    # merge genre and NOgenre
    if df_NOgenre.empty:
        df_merged = df_genre_merged
    else:
        df_merged = df_genre_merged.append(df_NOgenre_merged)
    
    df_merged = df_merged.sort_index()

    return df_merged

def getDownloadUrlPage():
    content = driver.find_elements_by_tag_name('a')
    data = []
    for element in content:
            data.append(element.get_attribute("href"))
            
    df = pd.DataFrame(data)
    df = df[df[0].str.contains(sitePath + r"/torrent/.......$",na=False)]
    df = df.iloc[8:]
    df = df.rename(columns={0:"downloadurl"})
    return df

def getDownloadFile(url):
    driver.get(url)
    content = driver.find_elements_by_tag_name('a')
    data = []
    for element in content:
            data.append(element.get_attribute("href"))
            
    df = pd.DataFrame(data)
    df = df[df[0].str.contains(sitePath + r"/download.php[?id=]",na=False)]
    lnk = df.iloc[0][0]
    driver.get(lnk)

def processResults(df,daysOld):
    # Add new columns
    df["Size_Converted"] = np.where(df.Size_Lable == 'GB',df.Size, df.Size.apply(lambda x: float(x)* 0.001))
    df.Leachers = df.Leachers.apply(lambda x: float(x))
    df.Seeders = df.Seeders.apply(lambda x: float(x))
    df["Ratio"] = (df.Seeders / df.Leachers) *100

    # filter dates and sort
    today = pd.Timestamp('now').floor('D') + timedelta(days=1)
    df['Date'] = pd.to_datetime(df['Date'], errors='ignore')
    df = df[(df['Date'] > today-timedelta(days=daysOld)) & (df['Date'] <= today)]
    df = df.sort_values(by=['Name','Date'], ascending=False)
    df = df[["Name","Date","Time","Size","Size_Lable","Seeders","Leachers","Comments","Uploader","downloadurl"]]
    df = df.reset_index(drop=True)

    return df

In [ ]:
# loop through all pages
df_allpages = pd.DataFrame(columns=["index","Name","Date","Time","Size","Size_Lable","Seeders","Leachers","Comments","Uploader","downloadurl"])
for search in searchList:
    for x in range1(1,1):
        time.sleep(3)
        df0 = getTablefrom(search,x)
        df1 = getDownloadUrlPage().reset_index()
        df2 = pd.concat([df0, df1], axis=1,sort=False)
        df_allpages = pd.concat([df_allpages, df2], axis=0,sort=False)

In [ ]:
# prcocess the results
df_allpages = processResults(df_allpages,int(lastXDays))
print(df_allpages)

In [ ]:
# Download files
df_download_list = df_allpages.downloadurl.tolist()
for x in df_download_list:
    time.sleep(3)
    getDownloadFile(x)

In [ ]:
# driver.close()